In [4]:
from mindquantum.utils import random_circuit
from mindquantum.core.circuit import Circuit
from huawei_graph_tool import ConvertCircuit,my_random_circuit,CustomGraphDataset
import random,os
import networkx as nx
from torch_geometric.utils import train_test_split_edges
from torch_geometric.data import Data
from mindquantum.io.qasm import HiQASM
from mindquantum.algorithm.compiler import  DAGCircuit,DAGQubitNode,DAGNode

In [5]:
root = './VGAE_dataset'

# 创建数据集对象
dataset = CustomGraphDataset(root=root)
#以下报错 
data = dataset[0]

# 将图数据对象分割为训练集和测试集
data = train_test_split_edges(data)
print(dataset[0])

Data(x=[20, 22], edge_index=[2, 20])


/Users/yangjianfei/mac_vscode/华为 QAS 实习/huawei_graph_tool.py:168: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.data, self.slices = torch.load(self.processed_paths[0])


In [6]:
data

Data(x=[20, 22], val_pos_edge_index=[2, 1], test_pos_edge_index=[2, 2], train_pos_edge_index=[2, 34], train_neg_adj_mask=[20, 20], val_neg_edge_index=[2, 1], test_neg_edge_index=[2, 2])

In [ ]:
# 固定随机数种子
#random.seed(42)

random_gate_num = random.randint(10, 35)
random_seed = random.randint(0, 5000)
sd_rate = random.uniform(0.3, 0.8)
ctrl_rate = random.uniform(0.1, 0.2)
print(sd_rate,random_gate_num,sd_rate,ctrl_rate,random_seed)
cir = my_random_circuit(n_qubits=8,gate_num=random_gate_num,sd_rate=sd_rate,ctrl_rate=ctrl_rate,seed=random_seed)
X,A,G,P,Gate_name =  ConvertCircuit(circuit=cir)
    #print(X.shape,A.shape)

In [3]:
cir = my_random_circuit(n_qubits=8,gate_num=10,sd_rate=sd_rate,ctrl_rate=ctrl_rate,seed=random_seed)

In [ ]:
X

In [ ]:
ConvertCircuit(cir)

In [ ]:
import numpy as np
HIQASM = HiQASM()
for i in range(5000):
    random_gate_num = random.randint(10, 30)
    random_seed = random.randint(0, 5000)
    sd_rate = random.uniform(0.3, 0.8)
    ctrl_rate = random.uniform(0.1, 0.2)
    print(sd_rate,random_gate_num)
    cir = my_random_circuit(n_qubits=8,gate_num=random_gate_num,sd_rate=sd_rate,ctrl_rate=ctrl_rate,seed=random_seed)
    X,A,G,P,Gate_name =  ConvertCircuit(circuit=cir)
    np.savetxt(f'./VGAE_dataset/X/X_{i}.csv', X, delimiter=',', fmt='%d')
    np.savetxt(f'./VGAE_dataset/A/A_{i}.csv', A, delimiter=',', fmt='%d')
    
    cir_unbound = cir.apply_value(dict(zip(cir.params_name, np.random.uniform(0, 2*np.pi, len(cir.params_name)))))
    #circuit_str = HIQASM.to_file(f'./VGAE_dataset/cir_img/cir{i}.hiqasm', cir_unbound, version='0.2')


In [7]:
from mindquantum.core.gates import CNOT
cir = Circuit()
cir+=CNOT.on(1,2)